In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
from onsstove.mca import MCA

In [3]:
from decouple import config

In [4]:
nepal = MCA(project_crs=3857, cell_size=(1000, 1000))

nepal.set_postgres(db='nepal', POSTGRES_USER=config('POSTGRES_USER'), POSTGRES_KEY=config('POSTGRES_KEY'))

In [5]:
nepal.add_mask_layer(name='Country boundaries', layer_path='admin.npl_admbnda_adm0_nd_20201117', postgres=True)

In [6]:
nepal.add_layer(category='demand', name='Education facilities', layer_path='places.education_facilities', postgres=True,
                layer_type='vector', normalization='MinMax', inverse=True, 
                distance='proximity', distance_limit=10000)

nepal.add_layer(category='demand', name='Population', layer_path='data/population_npl_2018-10-01.tif',
                layer_type='raster', normalization='MinMax', inverse=False, 
                distance='log', base_layer=True, resample='sum')

In [7]:
nepal.add_layer(category='supply', name='Existent transmission substations', layer_path='power_network.substations_georeferenced', 
                postgres=True, layer_type='vector', query='status == "E"', normalization='MinMax', inverse=True, 
                distance='proximity', distance_limit=30000)

nepal.add_layer(category='supply', name='Existent (below 220V) HV lines', layer_path='power_network.hv_lines_georeferenced',
                postgres=True, layer_type='vector', query='voltage <= 220 & status =="E"', normalization='MinMax', inverse=True, 
                distance='proximity', distance_limit=30000)

In [8]:
nepal.add_layer(category='supply', name='LPG dealers', layer_path='data/lpg_main_dealers/lpg_main_dealers.shp',
                postgres=False, layer_type='vector', normalization='MinMax', inverse=True, 
                distance='travel_time')

nepal.layers['supply']['LPG dealers'].add_friction_raster(raster_path=r"data\2020_motorized_friction_surface\2020_motorized_friction_surface.geotiff")

In [9]:
nepal.mask_layers(datasets='all')
nepal.reproject_layers(datasets='all')

In [10]:
nepal.get_distance_rasters(datasets='all')

nepal.normalize_rasters(datasets='all')

In [11]:
nepal.save_datasets()